# Problem 1
For this problem you will be using the Python vehicle simulator in the following Python block. The Federal Test Procedure (city) and (highway) drive cycles are described as the speed (in miles per hour) at each second of the drive cycle. The two drive cycles are provided as citycycle.csv and highwaycycle.csv. You will need to complete this Python program to model the road load power requirements over the course of the cycle and answer the following questions about an average 2018 midsize vehicle weighing 1,474 kg, with a frontal area of 2.08 m2, coefficient of drag CD of 0.27 and tires with a rolling resistance of CRR = 0.0075.  Considering both drive cycles answer the following:
1. What is the peak braking power requirement for this vehicle over the drive cycle? At the peak braking power point, what is the fraction of the total negative inertial power due to the brakes, rolling resistance and aerodynamic drag?  
**YOUR_ANSWER HERE**
1. Over the full drive cycle, what is the total energy required to overcome the road load? What is the fraction of this energy due to inertia, rolling resistance and aerodynamic drag?  
**YOUR_ANSWER HERE**

**3. What is the amount of energy lost to braking? Does this equal the negative of inertial energy required to move the vehicle? Explain why it does or does not equal the inertial energy.**  
The amount of energy lost to braking was 2.36 MJ for the city cycle and 1.46 MJ for the highway cycle. It is not the negative of the inertial energy, which was 3.09 MJ and 3.43 MJ respectively, because the inertial energy is the energy the vehicle has when in motion which is different to energy lost to braking which represents the friction and heat energy created during braking. Furthermore, the vehicle can decelerate through coasting which reduces the energy lost to braking when compared to the inertial energy.

![Mass vs Aerodynamic Reduction Impact on Total Energy](mass_aerodynamic_reduction.png)

**4. Does a 6% reduction in mass or aerodynamic properties (cd*A) of the vehicle have a greater impact on total energy required to overcome the road load? In the context of the drive cycle explain why the reduction has a greater impact.**  
A 6% reduction in mass (1474.0 to 1385.56 kg) yields a greater reduction in total energy than a 6% reduction in the drag coefficient (0.27 to 0.2538). This is because a reduction in the drag coefficient only has an impact on the aerodynamic energy used to move a vehicle while a reduction weight reduces the required rolling and inertial energy used during both the city and highway drive cycles, subsequently yielding a greater reduction and impact on the total energy required to overcome the road load.

In [47]:
from __future__ import annotations
import csv
from typing import Optional, NamedTuple
    
# 
# EME 189L Homework Simulator
#

class VehicleGlider:
    # Constants and conversions
    MPH_TO_MPS      = 0.44704
    METERS_TO_MILES = 0.000621371
    GRAVITY         = 9.80665     # m/s^2
    AIR_DENSITY     = 1.3         # kg/m^3

    class PowerComponents(NamedTuple):
        total : float
        inertia : float
        rolling : float
        aero : float

    def __init__(self, mass : float, area : float, cd : float, crr : float):
        self.__mass = mass
        self.__area = area
        self.__cd = cd
        self.__crr = crr
        self.reset()

    def reset(self):
        self.__velocity = 0.0
        self.__distance = 0.0
        self.__max_braking_power = 0.0
        self.__max_braking_inertia_power = 0.0
        self.__max_braking_rolling_power = 0.0
        self.__max_braking_aero_power = 0.0
        self.__last_rolling_power = 0.0
        self.__last_aero_power = 0.0
        self.__braking_energy = 0.0
        self.__inertial_energy = 0.0
        self.__rolling_energy = 0.0
        self.__aero_energy = 0.0

    @property
    def mass(self) -> float:
        return self.__mass
    
    @property
    def area(self) -> float:
        return self.__area
    
    @property
    def cd(self) -> float:
        return self.__cd
    
    @property
    def crr(self) -> float:
        return self.__crr
    
    @property
    def velocity(self) -> float:
        return self.__velocity
    
    @property
    def velocity_mph(self) -> float:
        return self.__velocity / VehicleGlider.MPH_TO_MPS

    @property
    def distance(self) -> float:
        return self.__distance
    
    @property
    def distance_miles(self) -> float:
        return self.__distance * VehicleGlider.METERS_TO_MILES

    @property
    def max_braking_power(self) -> float:
        return self.__max_braking_power

    @property
    def max_braking_inertia_power(self) -> float:
        return self.__max_braking_inertia_power
    
    @property
    def max_braking_rolling_power(self) -> float:
        return self.__max_braking_rolling_power
    
    @property
    def max_braking_aero_power(self) -> float:
        return self.__max_braking_aero_power
    
    @property
    def braking_energy(self) -> float:
        return self.__braking_energy
    
    @property
    def inertial_energy(self) -> float:
        return self.__inertial_energy
    
    @property
    def rolling_energy(self) -> float:
        return self.__rolling_energy
    
    @property
    def aero_energy(self) -> float:
        return self.__aero_energy
    
    @property
    def total_energy(self) -> float:
        total_energy = self.inertial_energy + self.rolling_energy + self.aero_energy
        return total_energy
    
    # Calculates the expected power to provide the given acceleration consider delta energy over delta time
    def required_inertial_power(self, delta_time : float, speed : float) -> float:
        # PUT YOUR CODE HERE
        required_inertial_power = self.mass * ((speed - self.velocity) / delta_time) * ((speed + self.velocity) / 2) # self.velocity is previous speed
        return required_inertial_power
        # pass

    # Calculates the expected power to overcome rolling resistance at the new desired speed
    def required_rolling_power(self, speed : float) -> float:
        # PUT YOUR CODE HERE
        required_rolling_power = self.crr * self.mass * self.GRAVITY * speed # slide 15
        return required_rolling_power     # pass

    # Calculates the expected power to overcome aero resistance at the new desired speed
    def required_aero_power(self, speed : float) -> float:
        # PUT YOUR CODE HERE
        required_aero_power = 0.5 * self.cd * self.AIR_DENSITY * self.area * (speed ** 3)
        return required_aero_power     # pass

    # Calculates the expected power components given delta time and the new desired speed use the previous functions
    # Returns a VehicleGlider.PowerComponents() NamedTuple
    def calculate_required_power(self, delta_time : float, speed : float) -> VehicleGlider.PowerComponents:
        # PUT YOUR CODE HERE
        inertia = self.required_inertial_power(delta_time, speed)
        rolling = self.required_rolling_power(speed)
        aero = self.required_aero_power(speed)
        total = inertia + rolling + aero
        power_components = VehicleGlider.PowerComponents(total, inertia, rolling, aero)
        return power_components     # pass

    # Finds the resulting speed for a specifi    ower
    def calculate_final_speed(self, delta_time : float, power : float) -> float:
        current_speed = self.velocity
        current_power = self.calculate_required_power(delta_time=delta_time,speed=current_speed)
        if current_power.total == power:
            return self.velocity
        while power > current_power.total:
            current_speed += 1.0
            current_power = self.calculate_required_power(delta_time=delta_time,speed=current_speed)
        if current_power.total == power:
            return current_speed
        while power < current_power.total and current_speed:
            current_speed -= 1.0
            current_speed = max(current_speed,0.0)
            current_power = self.calculate_required_power(delta_time=delta_time,speed=current_speed)
        min_speed = current_speed
        max_speed = current_speed + 1.0
        while current_power.total != power and max_speed > min_speed:
            current_speed = (min_speed + max_speed) * 0.5
            if current_speed == min_speed or current_speed == max_speed:
                break
            current_power = self.calculate_required_power(delta_time=delta_time,speed=current_speed)
            if current_power.total > power:
                max_speed = current_speed
            elif current_power.total < power:
                min_speed = current_speed
            
        return current_speed
        
    # Simulates the next timestep for the vehicle. 
    def simulate_timestep(self, delta_time : float, power : float, speed : Optional[float] = None):
        if speed is None:
            # This if is used for problem 3
            speed = self.calculate_final_speed(delta_time=delta_time,power=power)
        
        required_power = self.calculate_required_power(delta_time, speed)

        # integrate power to get energy
        # self.__inertial_energy += abs(required_power.inertia) * delta_time # not meant to return abs
        
        if required_power.inertia > 0:
            self.__inertial_energy += required_power.inertia * delta_time

        self.__rolling_energy += required_power.rolling * delta_time
        self.__aero_energy += required_power.aero * delta_time
        
        # update total distance traveled
        self.__distance += speed * delta_time

        # update velocity for next timestep
        self.__velocity = speed

        if required_power.total < self.max_braking_power:
            self.__max_braking_power = required_power.total
            self.__max_braking_inertia_power = required_power.inertia
            self.__max_braking_rolling_power = required_power.rolling
            self.__max_braking_aero_power = required_power.aero
        
        # I'm not sure if this is the correct way to calculate braking energy
        if required_power.total < 0:
            self.__braking_energy += abs(required_power.total) * delta_time



def run_sim(cycle : str, mass : float = 1474.0, area : float = 2.08, drag : float = 0.27, rolling : float = 0.0075) -> float:
    vehicle = VehicleGlider(mass=mass,area=area,cd=drag,crr=rolling)
    with open(cycle,'r') as in_file:
        dict_reader = csv.DictReader(in_file,delimiter=',')
        row = next(dict_reader)
        # Input first row for time, this should be 0 with 0 speed
        last_time = float(row['time'])
        for row in dict_reader:
            cur_time = float(row['time'])
            cur_speed = float(row['speed_mph']) * VehicleGlider.MPH_TO_MPS
            delta_time = cur_time - last_time
            required_power = vehicle.calculate_required_power(delta_time=delta_time,speed=cur_speed)
            vehicle.simulate_timestep(delta_time=delta_time,power=required_power.total,speed=cur_speed)
            last_time = cur_time

    if vehicle.max_braking_inertia_power:
        braking_rolling_percent = -vehicle.max_braking_rolling_power/vehicle.max_braking_inertia_power 
        braking_aero_percent = -vehicle.max_braking_aero_power/vehicle.max_braking_inertia_power
        braking_power_percent = vehicle.max_braking_power/vehicle.max_braking_inertia_power
    else:
        braking_rolling_percent = 0.0
        braking_aero_percent = 0.0
        braking_power_percent = 0.0
    print (f'Maximum Braking Power Brakes:  {-vehicle.max_braking_power/1000.0:0.2f} kW')
    print (f'Maximum Braking Power Rolling: {braking_rolling_percent*100:0.1f}%')
    print (f'Maximum Braking Power Aero:    {braking_aero_percent*100:0.1f}%')
    print (f'Maximum Braking Power Brakes:  {braking_power_percent*100:0.1f}%')
    print (f'Distance Traveled:             {vehicle.distance_miles:0.2f} miles')
    print (f'Rolling Energy:                {vehicle.rolling_energy/1e6:0.2f} MJ ({vehicle.rolling_energy/vehicle.total_energy*100:0.1f}%)')
    print (f'Aero Energy:                   {vehicle.aero_energy/1e6:0.2f} MJ ({vehicle.aero_energy/vehicle.total_energy*100:0.1f}%)')
    print (f'Inertial Energy:               {vehicle.inertial_energy/1e6:0.2f} MJ ({vehicle.inertial_energy/vehicle.total_energy*100:0.1f}%)')
    print (f'Braking Energy:                {vehicle.braking_energy/1e6:0.2f} MJ')
    print (f'Total Energy:                  {vehicle.total_energy/1e6:0.2f} MJ')

    return vehicle.total_energy/1e6

Below is the block to run the City and the Highway cycles. Make sure to run the above block before running the following code.

In [48]:
print('----------------- City Cycle -----------------')
city_mj = run_sim('citycycle.csv')
print('--------------- Highway Cycle ----------------')
highway_mj = run_sim('highwaycycle.csv')

----------------- City Cycle -----------------
Maximum Braking Power Brakes:  25.38 kW
Maximum Braking Power Rolling: 5.0%
Maximum Braking Power Aero:    2.8%
Maximum Braking Power Brakes:  92.2%
Distance Traveled:             7.45 miles
Rolling Energy:                1.30 MJ (24.3%)
Aero Energy:                   0.96 MJ (17.9%)
Inertial Energy:               3.09 MJ (57.8%)
Braking Energy:                2.36 MJ
Total Energy:                  5.35 MJ
--------------- Highway Cycle ----------------
Maximum Braking Power Brakes:  35.85 kW
Maximum Braking Power Rolling: 4.8%
Maximum Braking Power Aero:    4.9%
Maximum Braking Power Brakes:  90.3%
Distance Traveled:             20.51 miles
Rolling Energy:                3.58 MJ (27.0%)
Aero Energy:                   6.23 MJ (47.1%)
Inertial Energy:               3.43 MJ (25.9%)
Braking Energy:                1.46 MJ
Total Energy:                  13.25 MJ


# Problem 2
For this problem you will be using the simulator code that you developed in Problem 1. Select a 2022 EV that is on the market and another conventional (or non-plugin hybrid) vehicle in the same vehicle size class (compact, midsize, etc.). Simulate the two vehicles using the simulator for problem 1. Assuming 121.3MJ/gallon of gasoline, compare the weighted EPA fuel economy (55% city + 45% highway) of the two vehicles and the results from the simulations. EVs may be listed in miles per gallon gasoline equivalent (mpgge or mpg-e). Answer the following questions:

**1. What is the difference in fuel economy between the simulated vehicles and the published EPA numbers? Explain why there is a difference between the results you have from simulation and the published EPA numbers.**  
Our electric car had a calculated 136.11 MPG-e compared to its actual EPA estimate of 109 MPG-e. Our internal combustion engine vehicle had a calculated 152.66 MPG versus its EPA estimated 28 MPG. There is a difference between our simulation results and the published EPA numbers because our simulation does not take into account the efficiency of the powertrains. Our EV had a calculated 108.89 MPG-e value, or within 0.1% of its actual EPA estimate, when accounting for an 80% motor efficiency. Similarly, our ICE MPG values were more reasonably similar with a calculated 38 MPG when accounting for the 25% efficiency value of an engine. The remaining difference between our calculated and EPA values can be attributed to how our simulation doesn’t account for other sources of energy loss in ICE vehicles including transmission losses and modeling non-linear relationships between power and fuel efficiency across different gears or with varying engine loads.

|                                     |                    2022 BMW i4 eDrive40e                   |                  2022 BMW 430i Gran Coupe                  |
|-------------------------------------|------------------------------------------------------------|------------------------------------------------------------|
| City MPG (55%)                      |  $\frac{7.45mi}{\frac{7.29MJ}{121.3 MJ/Gal}}=123.96 MPG$   |   $\frac{7.45mi}{\frac{6.23MJ}{121.3 MJ/Gal}}=145.05 MPG$  |
| Highway MPG (45%)                   |  $\frac{20.51mi}{\frac{16.48MJ}{121.3 MJ/Gal}}=150.96 MPG$ |  $\frac{20.51mi}{\frac{15.36MJ}{121.3 MJ/Gal}}=161.97 MPG$ |
| Combined MPG (calculated)           | 136.11 MPG-e                                               | 152.66 MPG                                                 |
| Combined MPG (calculated, adjusted) | 108.89 MPG-e                                               | 38.17 MPG                                                  |
| Combined MPG (EPA estimate)         | 109 MPG-e                                                  | 28 MPG                                                     |

**2. Using the published city and highway numbers for both vehicles, what do you notice about the difference between the city vs. highway of the conventional vehicle compared to the city vs. highway of the EV? How does the city correlate to the highway for the conventional vehicle and for the EV? Explain why the correlations are different.**  
The city and highway MPG for the EV is similar (109 MPG-e for city driving versus 108 MPG-e for highway driving), while the difference is significantly greater for the conventional vehicle (25 MPG for city driving versus 34 MPG for highway driving). This is because EV’s use brake energy regeneration to recuperate braking losses and regain charge, increasing the miles it can cover and making its efficiency similar to its highway MPG-e estimate. Comparatively, conventional non-hybrid ICE vehicles are not able to turn the lost braking energy back into gasoline and subsequently perform worse during the city cycle. This is in addition to the lack of instant maximum torque and the using lower gears to balance efficiency and initial acceleration from a stop, resulting in a larger disparity between its city and highway cycle EPA estimates.

In [49]:
# Replace with vehicle's parameters
v1_make_model : str = '2022 BMW i4 eDrive40e'
v1_mass : float = 2116.0
v1_area : float = 2.31
v1_drag : float = 0.25
v1_rolling : float = 0.0075
print(f'Simulation results for {v1_make_model}')
print('----------------- City Cycle -----------------')
v1_city_mj = run_sim('citycycle.csv',mass=v1_mass,area=v1_area,drag=v1_drag,rolling=v1_rolling)
print('--------------- Highway Cycle ----------------')
v1_highway_mj = run_sim('highwaycycle.csv',mass=v1_mass,area=v1_area,drag=v1_drag,rolling=v1_rolling)

# Replace with vehicle's parameters
v2_make_model : str = '2022 BMW 430i Gran Coupe'
v2_mass : float = 1720.0
v2_area : float = 2.31
v2_drag : float = 0.28
v2_rolling : float = 0.0075
print(f'Simulation results for {v2_make_model}')
print('----------------- City Cycle -----------------')
v2_city_mj = run_sim('citycycle.csv',mass=v2_mass,area=v2_area,drag=v2_drag,rolling=v2_rolling)
print('--------------- Highway Cycle ----------------')
v2_highway_mj = run_sim('highwaycycle.csv',mass=v2_mass,area=v2_area,drag=v2_drag,rolling=v2_rolling)

Simulation results for 2022 BMW i4 eDrive40e
----------------- City Cycle -----------------
Maximum Braking Power Brakes:  36.75 kW
Maximum Braking Power Rolling: 5.0%
Maximum Braking Power Aero:    2.0%
Maximum Braking Power Brakes:  93.0%
Distance Traveled:             7.45 miles
Rolling Energy:                1.87 MJ (25.6%)
Aero Energy:                   0.99 MJ (13.5%)
Inertial Energy:               4.44 MJ (60.9%)
Braking Energy:                3.47 MJ
Total Energy:                  7.29 MJ
--------------- Highway Cycle ----------------
Maximum Braking Power Brakes:  52.26 kW
Maximum Braking Power Rolling: 4.8%
Maximum Braking Power Aero:    3.5%
Maximum Braking Power Brakes:  91.7%
Distance Traveled:             20.51 miles
Rolling Energy:                5.14 MJ (31.2%)
Aero Energy:                   6.41 MJ (38.9%)
Inertial Energy:               4.93 MJ (29.9%)
Braking Energy:                2.30 MJ
Total Energy:                  16.48 MJ
Simulation results for 2022 BMW 430i Gr

# Problem 3
For this problem you will be using the simulator code below that utilizes the code from problem 1. The goal of this problem is to develop a driver (a controller) to drive the vehicle. Unlike the simulation in problem 1 the speed is not input to the vehicle model, but only a power output. The driver will be initialized with the control frequency (assume at least 10Hz), the 0 – 60 mph time, the maximum power output, and percent of max power needed to cruise at 60 mph. The 0 – 60 mph time and the maximum power are values that a driver could be provided before driving the vehicle. Additionally, driving the vehicle, the driver could determine how much accelerator pedal is needed to maintain 60 mph. For each time step of the simulation the driver will need to:
1. Calculate the vehicle power given current vehicle speed, target vehicle speed, and a five seconds of time step look ahead.
1. The driver must maintain &#177;1 mph of the target speed. (Deviating from the &#177;1 mph invalidates EPA fuel economy tests.)
1. The driver must not request more power than the max power provided at initialization.
The driver must function on both driving cycles and for vehicles with different parameters. An output CSV file is generated showing the target, min, max, and actual vehicle speed for the drive cycle. This could be helpful if trying to visualize the behavior of the driver. The teams will be compared to one another, with a goal of providing a driver with the best combined fuel economy.

Helpful Hints:
1. The lookahead values could be useful but are not necessary to create an accurate driver.
1. Some form a feed forward control may be helpful when trying to develop the driver.
1.	The power required can be thought of a scaling of the max power. Below shows the breakdown, remember that the percent to hold at 60mph is the aero and rolling at 60mph. The other potion is the required power to increase the mass of the vehicle to 60mph in the 0-60 time.
1.	Once you have some idea of scaling, you will have a feed forward ballpark amount. You can use the feed forward plus some amount of feedback to calculate the required values. One of the easiest ways is to use PID. There are a lot of ways to calculate the error, but one way is to look at the target speed and the current speed. Another could delay a cycle by looking at the previous target speed and the current speed (how well did you do at meeting the desired target).
1.	Tips on tuning PID: It is best to just start with proportional. Integral will help remove steady state error. One gotcha with integral is that the integrated error could potentially spool up (consider situation where desired position is slightly passed some hard limit, so actually where want to be, but error keeps integrating). The spooling up of integral is probably not a huge issue due to the nature of the system. Only add in derivative if you need better response, you may be able to get good results with PI alone.

![Max Power Breakdown](max_power_fig.png)

In [50]:
from typing import List, NamedTuple
class TimeStep(NamedTuple):
    timestamp : float
    speed : float

class Driver:

    def __init__(self, freq : int, zero_sixty_time : float, max_power : float, cruise_60mph_percent : float):
        self.__control_frequency = freq
        self.__zero_sixty_time = zero_sixty_time
        self.__max_power = max_power
        self.__cruise_60mph_percent = cruise_60mph_percent
        # PUT IN MORE PARMETERS IF DESIRED
        self.__inertial_power = max_power * (1 - cruise_60mph_percent)
        self.__aero_rolling_power = max_power * cruise_60mph_percent
        self.__aero_power = self.__aero_rolling_power * 0.70
        self.__rolling_power = self.__aero_rolling_power * 0.30
        self.__sixty_mps = 60.0 * VehicleGlider.MPH_TO_MPS
        self.__acceleration = (self.__sixty_mps/self.__zero_sixty_time)
        self.__mass = (self.__inertial_power)/(self.__acceleration * (self.__sixty_mps/2))

        self.__a = self.__aero_power/(self.__sixty_mps ** 3)
        self.__b = self.__rolling_power/(self.__sixty_mps)
        print("Inertial Power: " + str(self.__inertial_power))
        print("Aero Power: " + str(self.__aero_power))
        print("Rolling Power: " + str(self.__rolling_power))
        print("a: " + str(self.__a))
        print("b: " + str(self.__b))
        print("Mass: " + str(self.__mass))
                

    @property
    def control_frequency(self) -> int:
        return self.__control_frequency

    @property
    def max_power(self) -> float:
        return self.__max_power
    
    @property
    def zero_sixty_time(self) -> float:
        return self.__zero_sixty_time
    
    @property
    def cruise_60mph_percent(self) -> float:
        return self.__cruise_60mph_percent
    
    @property
    def inertial_power(self) -> float:
        return self.__inertial_power

    # Calculates the desired power given the current speed, target speed and time to get to target
    def control(self, current_speed : float, target_speed : float, lookahead_timesteps : List[TimeStep]) -> float:
        # PUT YOUR CODE HERE
        eff_target_speed = lookahead_timesteps[2].speed
        error = ((eff_target_speed - current_speed)) * VehicleGlider.MPH_TO_MPS
        inertial_power = self.__mass * ((eff_target_speed - current_speed)/1) * error
        aero_power = self.__a * ((eff_target_speed * VehicleGlider.MPH_TO_MPS) ** 3)
        rolling_power = self.__b * (eff_target_speed * VehicleGlider.MPH_TO_MPS)
        total_power = (inertial_power + aero_power + rolling_power)
        print("--------------------------------------------------------------------")
        print("Error: " + str(error))
        print("Target Speed: " + str(target_speed))
        print("Current Speed: " + str(current_speed))
        print("Inertial Power: " + str(inertial_power))
        print("Aero Power: " + str(aero_power))
        print("Rolling Power: " + str(rolling_power))
        print("--------------------------------------------------------------------")
        if total_power > self.__max_power:
            return self.__max_power
        return total_power


    
def load_and_upscale_cycle(filename : str, freq : int) -> List[TimeStep]:
    return_list : List[TimeStep] = list()
    with open(filename,'r') as in_file:
        dict_reader = csv.DictReader(in_file,delimiter=',')
        row = next(dict_reader)
        # Input first row for time, this should be 0 with 0 speed
        last_time = float(row['time'])
        last_speed = float(row['speed_mph'])
        for row in dict_reader:
            cur_time = float(row['time'])
            cur_speed = float(row['speed_mph'])
            if cur_time - last_time != 1.0:
                raise ValueError(f'Time in {filename} does not increment by 1 second {last_time} -> {cur_time} has delta of {cur_time - last_time}.')
            for index in range(freq):
                return_list.append(TimeStep(timestamp=last_time + index / freq,speed=last_speed+(index/freq)*(cur_speed-last_speed)))
            last_time = cur_time
            last_speed = cur_speed
        return_list.append(TimeStep(timestamp=last_time,speed=last_speed))
    return return_list

def run_sim_driver(cycle : str, 
                    mass : float = 1474.0, 
                    area : float = 2.08, 
                    drag : float = 0.27, 
                    rolling : float = 0.0075,
                    freq : int = 10,
                    zero_sixty : float = 5.8,
                    output_filename : str = './out.csv') -> float:
    max_lookahead_time = 5

    vehicle = VehicleGlider(mass=mass,area=area,cd=drag,crr=rolling)
    # Provide driver with some scaling for power
    required_power_0_60mph = vehicle.calculate_required_power(delta_time=zero_sixty, speed=60.0 * VehicleGlider.MPH_TO_MPS)
    driver = Driver(freq=freq, zero_sixty_time=zero_sixty, max_power=required_power_0_60mph.total, cruise_60mph_percent=(required_power_0_60mph.aero + required_power_0_60mph.rolling) / required_power_0_60mph.total)
    cycle = load_and_upscale_cycle(filename=cycle,freq=freq)
    with open(output_filename,'w') as out_file:
        csv_writer = csv.writer(out_file,quoting=csv.QUOTE_ALL)
        csv_writer.writerow(['time','target','actual','min','max'])
        last_time_speed = cycle[0]
        csv_writer.writerow([last_time_speed.timestamp,last_time_speed.speed,0.0,max(last_time_speed.speed-1.0,0),last_time_speed.speed+1.0])
        for index,cur_time_speed in enumerate(cycle):
            if last_time_speed.timestamp < cur_time_speed.timestamp:
                lookahead_index = index + freq * max_lookahead_time if index + freq * max_lookahead_time < len(cycle) else None
                delta_time = cur_time_speed.timestamp - last_time_speed.timestamp
                required_power = driver.control(current_speed=vehicle.velocity_mph,target_speed=cur_time_speed.speed,lookahead_timesteps=cycle[index:lookahead_index])
                vehicle.simulate_timestep(delta_time=delta_time,power=required_power)
                csv_writer.writerow([cur_time_speed.timestamp,cur_time_speed.speed,vehicle.velocity_mph,max(cur_time_speed.speed-1.0,0),cur_time_speed.speed+1.0])
                if cur_time_speed.speed - 1.0 > vehicle.velocity_mph or cur_time_speed.speed + 1.0 < vehicle.velocity_mph:
                    print(f'Driver failed test cycle at {cur_time_speed.timestamp:.1f}s with vehicle exceeding +- 1 mph of {cur_time_speed.speed:.2f} with {vehicle.velocity_mph:.2f}.')
                    return -1
            last_time_speed = cur_time_speed

    if vehicle.max_braking_inertia_power:
        braking_rolling_percent = -vehicle.max_braking_rolling_power/vehicle.max_braking_inertia_power 
        braking_aero_percent = -vehicle.max_braking_aero_power/vehicle.max_braking_inertia_power
        braking_power_percent = vehicle.max_braking_power/vehicle.max_braking_inertia_power
    else:
        braking_rolling_percent = 0.0
        braking_aero_percent = 0.0
        braking_power_percent = 0.0
    print (f'Maximum Braking Power Brakes:  {-vehicle.max_braking_power/1000.0:0.2f} kW')
    print (f'Maximum Braking Power Rolling: {braking_rolling_percent*100:0.1f}%')
    print (f'Maximum Braking Power Aero:    {braking_aero_percent*100:0.1f}%')
    print (f'Maximum Braking Power Brakes:  {braking_power_percent*100:0.1f}%')
    print (f'Distance Traveled:             {vehicle.distance_miles:0.2f} miles')
    print (f'Rolling Energy:                {vehicle.rolling_energy/1e6:0.2f} MJ ({vehicle.rolling_energy/vehicle.total_energy*100:0.1f}%)')
    print (f'Aero Energy:                   {vehicle.aero_energy/1e6:0.2f} MJ ({vehicle.aero_energy/vehicle.total_energy*100:0.1f}%)')
    print (f'Inertial Energy:               {vehicle.inertial_energy/1e6:0.2f} MJ ({vehicle.inertial_energy/vehicle.total_energy*100:0.1f}%)')
    print (f'Braking Energy:                {vehicle.braking_energy/1e6:0.2f} MJ')
    print (f'Total Energy:                  {vehicle.total_energy/1e6:0.2f} MJ')

    return 0

Use the block below to run the driver simulation.

In [51]:
for drive_cycle_name in ['citycycle.csv', 'highwaycycle.csv']:
    output_filename = f'./out_{drive_cycle_name}'
    print(f'----------------- {drive_cycle_name} -----------------')
    run_sim_driver(cycle=drive_cycle_name,output_filename=output_filename)
    print('')

----------------- citycycle.csv -----------------
Inertial Power: 91418.6416339862
Aero Power: 6966.477803213165
Rolling Power: 2985.633344234214
a: 0.36101093198711165
b: 111.31119304142112
Mass: 1474.0
--------------------------------------------------------------------
Error: 0.0
Target Speed: 0.0
Current Speed: 0.0
Inertial Power: 0.0
Aero Power: 0.0
Rolling Power: 0.0
--------------------------------------------------------------------
--------------------------------------------------------------------
Error: 0.0
Target Speed: 0.0
Current Speed: 0.0
Inertial Power: 0.0
Aero Power: 0.0
Rolling Power: 0.0
--------------------------------------------------------------------
--------------------------------------------------------------------
Error: 0.0
Target Speed: 0.0
Current Speed: 0.0
Inertial Power: 0.0
Aero Power: 0.0
Rolling Power: 0.0
--------------------------------------------------------------------
--------------------------------------------------------------------
Erro